Join original 1000 random samples with entire df, then export again with added columns to process the remaining

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("../filtered_pension_amounts_subset.parquet")

In [3]:
df.shape

(201523, 7)

In [4]:
df_1000_sample = pd.read_json("./extracted_amounts_sample_1000_without_text.json")

In [5]:
df_1000_sample.shape

(872, 7)

In [71]:
# df_1000_sample.head()

In [6]:
df_1000_sample.columns = df_1000_sample.columns.str.replace(
    'extracted_pension_amount', 
    'llm_extracted_pension_amount'
)

In [73]:
# df_1000_sample.head()

In [7]:
df_1000_sample['llm_extracted_pension_amount_dollars'] = True

In [8]:
df_1000_sample.head()

,NAID,naraURL,title,pageObjectId,pageURL,file_cat,llm_extracted_pension_amount,llm_extracted_pension_amount_dollars
0,111708949,https://catalog.archives.gov/id/111708949,Revolutionary War Pension and Bounty Land Warr...,111708987,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,widow,"{'applicant_name': 'Eunice Wilder', 'applicant...",True
1,196100358,https://catalog.archives.gov/id/196100358,Revolutionary War Pension and Bounty Land Warr...,196100360,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,soldier,"{'applicant_name': 'George Keller', 'applicant...",True
2,111503547,https://catalog.archives.gov/id/111503547,Revolutionary War Pension and Bounty Land Warr...,111503552,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,widow,"{'applicant_name': 'Mildred Hart', 'applicant_...",True
3,54362674,https://catalog.archives.gov/id/54362674,Revolutionary War Pension and Bounty Land Warr...,54362689,https://s3.amazonaws.com/NARAprodstorage/opast...,widow,"{'applicant_name': 'Mary Date', 'applicant_typ...",True
4,54330151,https://catalog.archives.gov/id/54330151,Revolutionary War Pension and Bounty Land Warr...,54330216,https://s3.amazonaws.com/NARAprodstorage/opast...,widow,"{'applicant_name': 'John Crawford', 'applicant...",True


In [9]:
df_1000_sample.to_csv("extracted_amounts_sample_1000_pre_normalization.csv")

In [76]:
df_1000_sample.columns

Index(['NAID', 'naraURL', 'title', 'pageObjectId', 'pageURL', 'file_cat',
       'llm_extracted_pension_amount', 'llm_extracted_pension_amount_dollars'],
      dtype='object')

In [77]:
df.columns

Index(['NAID', 'naraURL', 'title', 'pageObjectId', 'pageURL', 'file_cat',
       'priority_text'],
      dtype='object')

In [78]:
# Ensure both dataframes have the same data types for the join columns
df['pageObjectId'] = df['pageObjectId'].astype(str)
df['NAID'] = df['NAID'].astype(str)
df_1000_sample['pageObjectId'] = df_1000_sample['pageObjectId'].astype(str)
df_1000_sample['NAID'] = df_1000_sample['NAID'].astype(str)

# Merge on both pageObjectId and NAID
df_merged = df.merge(
    df_1000_sample[['pageObjectId', 'NAID', 'llm_extracted_pension_amount', 'llm_extracted_pension_amount_dollars']], 
    on=['pageObjectId', 'NAID'], 
    how='left'
)

In [79]:
df.shape

(201523, 7)

In [80]:
df_merged.shape

(201523, 9)

In [81]:
df_merged.head()

,NAID,naraURL,title,pageObjectId,pageURL,file_cat,priority_text,llm_extracted_pension_amount,llm_extracted_pension_amount_dollars
0,111769418,https://catalog.archives.gov/id/111769418,Revolutionary War Pension and Bounty Land Warr...,111769420,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,soldier,1831\nto\nSchaled Tucker\n1941\nMassachusetts\...,NaN,NaN
1,111769418,https://catalog.archives.gov/id/111769418,Revolutionary War Pension and Bounty Land Warr...,111769422,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,soldier,"ESSEX, ss.\nCommonwealth of Massachusetts.\nAt...",NaN,NaN
2,111769418,https://catalog.archives.gov/id/111769418,Revolutionary War Pension and Bounty Land Warr...,111769424,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,soldier,and at the same time he produces to the Court ...,NaN,NaN
3,111769405,https://catalog.archives.gov/id/111769405,Revolutionary War Pension and Bounty Land Warr...,111769407,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,soldier,Vermont 18.682\n---\nPeter Woodbury\nof Windso...,NaN,NaN
4,111769387,https://catalog.archives.gov/id/111769387,Revolutionary War Pension and Bounty Land Warr...,111769390,https://s3.amazonaws.com/NARAprodstorage/lz/mi...,widow,"Massachusetts 1,25\nMary Woodbury\nwidow of Na...",NaN,NaN


In [82]:
print((df_merged['llm_extracted_pension_amount_dollars'] == True).sum())

893


In [ ]:
# Drop duplicates based on specific columns (excluding dict columns)
df_merged = df_merged.drop_duplicates(subset=['pageObjectId', 'NAID', 'title', 'naraURL', 'llm_extracted_pension_amount_dollars'])

In [85]:
print((df_merged['llm_extracted_pension_amount_dollars'] == True).sum())

872


In [86]:
df_merged.to_parquet('filtered_pension_amounts_subset_with_llm_1000.parquet')